In [ ]:
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import pandas as pd
import os
import numpy as np


indir='./min_basal_xyz/'
outdir='./out_run/'

for root, dirs, filenames in os.walk(indir):
    for n in filenames:
        print(n[:-4])
        if outdir == './out_run/':
            frame=str(253)
        if outdir == './out_c/':     
            frame=str(164)
        traj=open(outdir+n[:-4]+'/traj.in', 'w')
        traj.write('parm pcbm_LIG2.prmtop\n')
        traj.write('trajin traj.nc '+frame+' last 10\n')
        traj.write('trajout traj.mdcrd nobox\n')
        traj.write('run\n')
        traj.close()
        cmd='cd '+outdir+n[:-4]+'/ && cpptraj -i traj.in && rm traj.in'
        os.system(cmd)
        
        #mda.core.flags['use_pbc'] = False
        
        u = mda.Universe(outdir+n[:-4]+"/pcbm_LIG2.prmtop",outdir+n[:-4]+"/traj.mdcrd")
    
        
        
        sel_aceptor = "(resname PCB and not name H* C11 C10 C9 C8 C12 C7 C6 C1 C2 O1 O2 C3 C5 C4)"

        




        aceptor = u.select_atoms(sel_aceptor)

        
        

        
        aceptor_list=list(aceptor.residues)
    
        
#######




########
        

        pdb = mda.Writer(outdir+n[:-4]+"/all.pdb", multiframe=True)
        beta=[]
        beta2=[]
        save=[]
        larger=[]
        for ts in u.trajectory:
            x=[]
            for l in aceptor_list:
                    selection=aceptor.select_atoms("resname PCB and not name H* C11 C10 C9 C8 C12 C7 C6 C1 C2 O1 O2 C3 C5 C4 and (around 4.0 (resid "+str(l.id)+"))",pbc=False)
                    selection_list=list(selection.residues)
                  
                    x.append(selection_list)
            
        
            group=[]
            for j in aceptor_list:
                sub_group=[]
                sub_group.append(j.id)
                for i,line in enumerate(x):
                    if j in line:
                        i=i+1
                        sub_group.append(i)
                sub_group=set(sub_group)
                sub_group= sorted(sub_group)
                sub_group=list(sub_group)
                
                sub_group2=[]        
                for k in sub_group:
                    k=k-1
                    donor=aceptor_list[k]
                    for m,line in enumerate(x):
                        if donor in line:
                            m=m+1
                            sub_group2.append(m)
                sub_group2=set(sub_group2)
                sub_group2= sorted(sub_group2)
                sub_group2=list(sub_group2)
                group.append(sub_group2)


                    

        
        
        
        
            count=0
            while count< len(group):
                count=count+1
                grupo_conjunto=[]
                for i,j in enumerate(group):
                    i=i+1
                    conjunto=[]
                    conjunto.append(i)
                    prueba=set()
                    for  a,b in enumerate(group):
                            a=a+1
                            if i != a:
                                intersection= set(j) & set(b)
                                if intersection != prueba:
                                    conjunto.append(a)
                    conjunto=set(conjunto)
                    conjunto=sorted(conjunto)
                    conjunto=list(conjunto)
                    grupo_conjunto.append(conjunto)
                group=grupo_conjunto
                
        
            for i,line in enumerate(group):
                line=set(line)
                line=sorted(line)
                line=list(line)
                group[i]=" ".join(str(x) for x in line)
        
        
            group=set(group)
            group=list(group)
            
            res=u.select_atoms('resname PCB and not name H* C11 C10 C9 C8 C12 C7 C6 C1 C2 O1 O2 C3 C5 C4')
            res=list(res.atoms.residues)
            numero= res[0].select_atoms('resname PCB and not name H* C11 C10 C9 C8 C12 C7 C6 C1 C2 O1 O2 C3 C5 C4')
            numero=len(numero)
            res=len(res)
            
        
            save.append(len(group)/res)  
            ensayo=[]
            for k,i in enumerate(group):
                j=i.split(' ')
                #print(k, j)
                ensayo.append(j)
            #print(ensayo)
            large_proof=[]
            for i in ensayo:
                large_proof.append(len(i))
            sum_min=[]
            max_max=[]
            for i in large_proof:
                if i == min(large_proof):
                    sum_min.append(i)
                if i == max(large_proof):
                    max_max.append(i)
            sum_min=sum(sum_min)
            max_max=sum(max_max)
            
            larger.append(max(large_proof)/res)
            
            
            

            factor=[]
            for i in range(1,res+1):
                for l,j in enumerate(ensayo):
                    #print(len(j))
                    for k in j:
                        if float(k) == float(i):
                            m=l
                            for r in range(0,numero): ### átomos de la molécula 
                                    factor.append(m*10)
        
            
            
            
            aceptor3=u.select_atoms('resname PCB and not name H* C11 C10 C9 C8 C12 C7 C6 C1 C2 O1 O2 C3 C5 C4')
            factor=np.array(factor)
            #print(factor)
            pdb.write(aceptor3.atoms)
            aceptor3.atoms.bfactors=factor
            beta.append(factor)
            
            
             
            
        beta_max=max(save)
        pdb.close()
        save=pd.Series(save)
        if os.path.isfile(outdir+n[:-4]+'/cluster_aceptor.txt'):
            os.remove(outdir+n[:-4]+'/cluster_aceptor.txt')          
        save.to_csv(outdir+n[:-4]+'/cluster_aceptor.txt',index=False, mode='a')
        
        larger=pd.Series(larger)
        if os.path.isfile(outdir+n[:-4]+'/max.txt'):
            os.remove(outdir+n[:-4]+'/max.txt')          
        larger.to_csv(outdir+n[:-4]+'/max.txt',index=False, mode='a')
        
        #for k,i in enumerate(group):
        #    j=i.split(' ')
                
                
        #    print(k)
        #    print('cluster=',len(j))
        #    print(group)
        # serialize: add a marker 'END' after each frame
        userdata = "all.dat"
        vmdscript = "all.vmd"
        
        marker = 'ENDMDL'
        with open(outdir+n[:-4]+'/'+userdata, 'w') as data:
            for yy in beta:
                data.write("\n".join([str(x) for x in yy]))
                data.write("\n{0}\n".format(marker))
        data.close()

        # write VMD loader script
        parameters = {'datafile': userdata,
                      'molecule': 'all.pdb'}

        script = """\
        proc loaduserdata { fname } {
            set all [atomselect top all]
            set frame 0
            set data [open $fname r]
            while { [gets $data line] != -1 } {
                set value [string trim $line]
                switch -- [string range $value 0 3] {
                    ENDM {
                        $all frame $frame
                        $all set user $beta
                        set beta {}
                        incr frame
                    }
                    default {
                        lappend beta $line
                    }
                }
            }
        }
        """ + """
        mol new "{0[molecule]}" waitfor all
        loaduserdata "{0[datafile]}"
        mol modcolor 0 top User
        mol modstyle 0 top VDW
        """.format(parameters)+"""
        mol scaleminmax top 0 auto """

        with open(outdir+n[:-4]+'/'+vmdscript, 'w') as tcl:
            tcl.write(script+'\n')
        tcl.close()


        
        cmd='cd '+outdir+n[:-4]+'/ && rm traj.mdcrd && rm output.prmtop'
        os.system(cmd)

In [ ]:
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import pandas as pd
import os
from rdkit import Chem




scan='./min_basal_xyz/'
indir='./min_basal_mol/'
outdir='./out_run/'
numdir='./numbering_dihedral/'
mol2dir='./mol2_basal/'

for root, dirs, filenames in os.walk(scan):
    for n in filenames:
        print(n)
        if n[-10:-4] in ['_hexil']:
            m = Chem.MolFromMolFile(indir+n[:-10]+'.mol',removeHs=True)
        else:
            m = Chem.MolFromMolFile(indir+n[:-4]+'.mol',removeHs=True)
        bond_idx=[]
        
        r7=open(numdir+n[:-4]+'.dat', 'r')
        table7 = [line.rstrip().split('-') for line in r7.readlines()]
        if n[:-4] in ['C342','C385']:
            for line in range(0,int(len(table7)/4)):
                bond_idx.append(m.GetBondBetweenAtoms(int(table7[line*4][1])-2,int(table7[line*4][2])-2).GetIdx())
        
        else:
            for line in range(0,int(len(table7)/4)):
                bond_idx.append(m.GetBondBetweenAtoms(int(table7[line*4][1])-1,int(table7[line*4][2])-1).GetIdx())
                    
                    
                    
      
        fragmented_mol = Chem.FragmentOnBonds(m, bond_idx,addDummies=False)    
        #print(fragmented_mol)
        smile=Chem.MolToSmiles(fragmented_mol)
        
        table7=smile.split('.')
        
        
        NO=[]
        MM=[]
        FRAG=[]
        for index,i in enumerate(table7):
            
            m1=Chem.MolFromSmiles(table7[index])
            MM.append(m.GetSubstructMatches(m1))
        for i in MM:
            for j in i:
                if j not in FRAG:
                    FRAG.append(j)
        
        FRAG1=[]
        for i in range(0,len(FRAG)):
            for j in range(0,len(FRAG)):
                if j>i:
                    if set(FRAG[i]) & set(FRAG[j]) != set():
                        if len(FRAG[i])>len(FRAG[j]):
                            FRAG1.append(FRAG[j])
        FRAG3=[] 
        for i in FRAG:
                if i not in FRAG1:
                    FRAG3.append(i)
        
        
        NAMES=[]
        
        for j in FRAG3:
            names=[]
            lista=[]
            for l in j:
                lista.append(l)
            for i in lista:
                mol2file=open(mol2dir+n[:-4]+'.mol2','r')
                for line in mol2file:
                    if len(line)>70:
                        a=[line.rstrip().split(" ") for line in line]
                        b=str(a[4][0]+a[5][0]+a[6][0])
                        if str(i+1)==b:
                            names.append(str(a[8][0]+''+a[9][0]+''+a[10][0]+''+a[11][0]))
            NO.append(names)                           
            no=' '.join(names)+' '
            NAMES.append(no)
        print(NAMES)

        
        
        if outdir == './out_run/':
            frame=str(253)
        if outdir == './out_c/':     
            frame=str(164)
        traj=open(outdir+n[:-4]+'/traj.in', 'w')
        traj.write('parm pcbm_LIG2.prmtop\n')
        traj.write('trajin traj.nc '+frame+' last 10\n')
        traj.write('trajout traj.mdcrd nobox\n')
        #traj.write('trajout traj.mdcrd\n')
        traj.write('run\n')
        traj.close()
        cmd='cd '+outdir+n[:-4]+'/ && cpptraj -i traj.in && rm traj.in'
        os.system(cmd)

        
        u = mda.Universe(outdir+n[:-4]+"/pcbm_LIG2.prmtop",outdir+n[:-4]+"/traj.mdcrd")
        
        SELE=[]
        for i in NAMES:
            SELE.append("resname LIG and name "+str(i)+" ")
        
        data="resname LIG and name "+" ".join([ str(x) for x in NAMES])
        data2="name "+" ".join([ str(x) for x in NAMES])
        
        donor = u.select_atoms(data)
        
       
        donor_list=list(donor.residues)
        
                
        pdb = mda.Writer(outdir+n[:-4]+"/all1.pdb", multiframe=True,bonds=None)
        beta=[]
        save=[]
        larger=[]
        for ts in u.trajectory:
            
            
            #print(ts)
                
            donor = u.select_atoms(data)    
            donor_list=list(donor.residues)

            
        
        
            x=[]
            for l in donor_list:
                    selection=donor.select_atoms(data+" and (around 4.0 ("+data2+" and resid "+str(l.id)+"))",pbc=True)
        
                    selection_list=list(selection.residues)                 
                    x.append(selection_list)

            aa_n=u.select_atoms('resname PCB')
            aa_n=list(aa_n.residues)
            aa_n=len(aa_n)
            group=[]
            for j in donor_list:

                sub_group=[]
                sub_group.append(j.id)
                for i,line in enumerate(x):
                    if j in line:
                        i=i+aa_n+1
                        sub_group.append(i)
                sub_group=set(sub_group)
                sub_group= sorted(sub_group)
                sub_group=list(sub_group)
                sub_group2=[]        
                for k in sub_group:
                    k=k-aa_n-1
                    donor=donor_list[k]
                    #print(donor)
                    for m,line in enumerate(x):
                        if donor in line:
                            m=m+aa_n+1
                            sub_group2.append(m)
                sub_group2=set(sub_group2)
                sub_group2= sorted(sub_group2)
                sub_group2=list(sub_group2)
                group.append(sub_group2)

                    

        
        
            
            
            count=0
            while count< len(group):
                count=count+1
                grupo_conjunto=[]
                for i,j in enumerate(group):
                    i=i+aa_n+1
                    conjunto=[]
                    conjunto.append(i)
                    prueba=set()
                    for  a,b in enumerate(group):
                            a=a+aa_n+1
                            if i != a:
                                
                                intersection= set(j) & set(b)
                            #print(intersection)
                                if intersection != prueba:
                                    conjunto.append(a)
                    conjunto=set(conjunto)
                    conjunto=sorted(conjunto)
                    conjunto=list(conjunto)
                    grupo_conjunto.append(conjunto)
                group=grupo_conjunto
            
        
            for i,line in enumerate(group):
                line=set(line)
                line=list(line)
                group[i]=" ".join(str(x) for x in line)
        
        
            group=set(group)
            group=list(group)
            
            res=u.select_atoms(data)
            res=list(res.atoms.residues)
            numero= res[0].select_atoms(data)
            numero=len(numero)
            
            #print(numero)
            res=len(res)
            #print(res)
            res1=u.select_atoms('resname PCB')
            res1=list(res1.atoms.residues)
            res1=len(res1)
            
            save.append(len(group)/res)
            
            ensayo=[]
            for k,i in enumerate(group):
                j=i.split(' ')
                ensayo.append(j)
            
            large_proof=[]
            sum_min=[]
            max_max=[]
            for i in ensayo:
                large_proof.append(len(i))
            for i in large_proof:
                if i == min(large_proof):
                    sum_min.append(i)
                if i == max(large_proof):
                    max_max.append(i)
            sum_min=sum(sum_min)
            max_max=sum(max_max)
            
            larger.append(max(large_proof)/res)
            
            factor=[]
            for i in range(res1+1,res1+1+res):
                for l,j in enumerate(ensayo):
                    #print(len(j))
                    for k in j:
                        if float(k) == float(i):
                            m=l
                            for r in range(0,numero): ### átomos de la molécula 
                                    factor.append(m*10)
            
            aceptor3=u.select_atoms(data)
            factor=np.array(factor)
            aceptor3.atoms.bfactors=factor
            #print(len(factor))
            pdb.write(aceptor3.atoms)
            beta.append(factor)
            
            
        beta_max=max(save)    
        pdb.close()
        save=pd.Series(save)
        #print(save)
        if os.path.isfile(outdir+n[:-4]+'/cluster_donor.txt'):
            os.remove(outdir+n[:-4]+'/cluster_donor.txt')          
        save.to_csv(outdir+n[:-4]+'/cluster_donor.txt',index=False, mode='a')
        
        larger=pd.Series(larger)
        if os.path.isfile(outdir+n[:-4]+'/maxl.txt'):
            os.remove(outdir+n[:-4]+'/maxl.txt')          
        larger.to_csv(outdir+n[:-4]+'/maxl.txt',index=False, mode='a')

        
        userdata = "all1.dat"
        vmdscript = "all1.vmd"

        marker = 'ENDM'
        with open(outdir+n[:-4]+'/'+userdata, 'w') as data:
            for yy in beta:
                data.write("\n".join([str(x) for x in yy]))
                data.write("\n{0}\n".format(marker))
        data.close()

        # write VMD loader script
        parameters = {'datafile': userdata,
                      'molecule': 'all1.pdb'}

        script = """\
        proc loaduserdata { fname } {
            set all [atomselect top all]
            set frame 0
            set data [open $fname r]
            while { [gets $data line] != -1 } {
                set value [string trim $line]
                switch -- [string range $value 0 3] {
                    ENDM {
                        $all frame $frame
                        $all set user $beta
                        set beta {}
                        incr frame
                    }
                    default {
                        lappend beta $line
                    }
                }
            }
        }
        """ + """
        mol new "{0[molecule]}" waitfor all
        loaduserdata "{0[datafile]}"
        mol modcolor 0 top User
        mol modstyle 0 top VDW
        """.format(parameters)+"""
        mol scaleminmax top 0 auto """

        with open(outdir+n[:-4]+'/'+vmdscript, 'w') as tcl:
            tcl.write(script+'\n')
        tcl.close()


        
        cmd='cd '+outdir+n[:-4]+'/ && rm traj.mdcrd'
        os.system(cmd)